# Cedar TPE RAG Authorization Demo

This notebook demonstrates **authorization-scoped retrieval** in a RAG system using Cedar's type-aware partial evaluation (TPE).

## Overview

The pattern we're demonstrating:

1. **Cedar TPE** evaluates policies with a known principal and action, but unknown resource
2. Cedar produces a **residual policy** describing which resource attributes matter for access
3. The application **translates that residual into retrieval constraints** (OpenSearch filter)
4. Vector search runs **with those constraints applied**, so only authorized chunks can become LLM context

**Key insight:** Authorization is enforced *before* any sensitive context is assembled, not in the prompt.



## System Architecture

![Authorization-Scoped RAG Architecture](docs/RAG.png)

In a retrieval-augmented generation (RAG) system, the application enriches a user's prompt with additional context retrieved from a vector database—documents, passages, or chunks that help the language model produce a more informed response. In an authorized RAG system, that enrichment is not generic or unconditional. As the diagram shows, authorization is evaluated before retrieval, not inside the prompt. Cedar's type-aware partial evaluation evaluates policy for a given principal and action and produces a policy residual, a constraint over resources. The application compiles that residual into a database-native query filter, ensuring that only authorized additional context is retrieved from the vector database. The prompt constructor then assembles the user's instruction together with this authorized additional context. The language model never decides what it is allowed to see; it operates entirely within a world that has already been shaped by authorization.


## Prerequisites

Before running this notebook, ensure:

1. **Cedar CLI is installed** with experimental TPE feature enabled
   - See README.md for build instructions
   - Verify with: `cedar tpe --help`

2. **Node.js dependencies are installed:**
   ```bash
   npm install
   ```

3. **Output directory exists:**
   ```bash
   mkdir -p out
   ```

Let's verify the setup:


In [ ]:
import subprocess
import json
import os
from pathlib import Path

# Check if Cedar CLI is available
try:
    result = subprocess.run(['cedar', '--version'], capture_output=True, text=True, check=True)
    print(f"✓ Cedar CLI found: {result.stdout.strip()}")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("✗ Cedar CLI not found. Please install Cedar CLI with TPE feature enabled.")
    print("  See README.md for instructions.")

# Check if Node.js scripts exist
scripts = [
    'src/tpe/partial-eval.js',
    'src/compile/residual-to-filter.js'
]
for script in scripts:
    if Path(script).exists():
        print(f"✓ {script} found")
    else:
        print(f"✗ {script} not found")

# Ensure output directory exists
os.makedirs('out', exist_ok=True)
print(f"✓ Output directory ready")


✓ Cedar CLI found: cedar-policy-cli 4.8.2
✓ src/tpe/partial-eval.js found
✓ src/compile/residual-to-filter.js found
✓ Output directory ready


## Step 1: Understanding the Policy Model

Let's examine the Cedar schema, policies, and entities that define our authorization model.


In [ ]:
# Read and display the Cedar schema
with open('cedar/schema.cedarschema', 'r') as f:
    schema = f.read()
print("=== Cedar Schema ===")
print(schema)


=== Cedar Schema ===
namespace Platform {
  entity Tenant;

  entity Team {
    tenant: Tenant,
  };

  // A person is either an Employee or a Customer.
  entity Employee {
    tenant: Tenant,
    teams: Set<Team>,
  };

  entity Customer {
    tenant: Tenant,
    teams: Set<Team>,
  };

  // Documents belong to a tenant and are shared via reader teams.
  entity Document {
    tenant: Tenant,

    // Reader teams may be null when a document is not shared with that category.
    // Note: Cedar doesn't support optional types directly, so these are always present but may be null
    employee_readers_team: Team,
    customer_readers_team: Team,

    // Simple sensitivity signal for the demo.
    classification: String,
  };

  // A chunk is the unit stored in the vector index.
  // It inherits tenant/sensitivity from the parent document.
  entity Chunk {
    tenant: Tenant,
    doc: Document,
    classification: String,

    // Flattened sharing attributes for easier pushdown filtering.
  

### Policy Overview

Our demo includes four policies:

1. **`tenant-scope`**: Enforces multi-tenancy - principals can only access resources in their own tenant
2. **`customer-view`**: Allows customers to view chunks shared with their customer reader team
3. **`employee-view`**: Allows employees to view chunks shared with their employee reader team
4. **`classification-limit`**: Forbids customers from accessing confidential resources (employees can access confidential)

Let's look at the policies:


In [ ]:
# List and display all policies
policy_files = sorted(Path('cedar/policies').glob('*.cedar'))
print(f"Found {len(policy_files)} policies:\n")

for policy_file in policy_files:
    print(f"=== {policy_file.name} ===")
    with open(policy_file, 'r') as f:
        print(f.read())
    print()


Found 4 policies:

=== classification-limit.cedar ===
@id("classification-limit")
forbid(
  principal is Platform::Customer,
  action in [Platform::Action::"view", Platform::Action::"ask"],
  resource is Platform::Chunk
)
when {
  resource.classification == "confidential"
};

=== customer-view.cedar ===
@id("customer-view")
permit(
  principal is Platform::Customer,
  action in [Platform::Action::"view", Platform::Action::"ask"],
  resource is Platform::Chunk
)
when {
  principal.teams.contains(resource.customer_readers_team)
};



=== employee-view.cedar ===
@id("employee-view")
permit(
  principal is Platform::Employee,
  action in [Platform::Action::"view", Platform::Action::"ask"],
  resource is Platform::Chunk
)
when {
  principal.teams.contains(resource.employee_readers_team)
};

=== tenant-scope.cedar ===
@id("tenant-scope")
permit(
  principal,
  action,
  resource
)
when {
  // All principals in this demo have a tenant attribute.
  principal.tenant == resource.tenant
};




### Entity Data

Let's examine the entities (principals and resources) in our demo:


In [ ]:
# Load and display entities
with open('cedar/entities.json', 'r') as f:
    entities = json.load(f)

# Group entities by type
by_type = {}
for entity in entities:
    entity_type = entity['uid']['type']
    if entity_type not in by_type:
        by_type[entity_type] = []
    by_type[entity_type].append(entity)

print("=== Entities by Type ===\n")
for entity_type, entity_list in sorted(by_type.items()):
    print(f"{entity_type}: {len(entity_list)} entities")
    for entity in entity_list:
        entity_id = entity['uid']['id']
        print(f"  - {entity_id}")
    print()


=== Entities by Type ===

Platform::Chunk: 2 entities
  - q3-plan#1
  - hr-note#1

Platform::Customer: 3 entities
  - kate
  - jack
  - mallory

Platform::Document: 2 entities
  - q3-plan
  - hr-note

Platform::Employee: 1 entities
  - alice

Platform::Team: 4 entities
  - custco-readers
  - custco-employees
  - otherco-readers
  - none

Platform::Tenant: 2 entities
  - custco
  - otherco



## Step 2: Type-Aware Partial Evaluation (TPE)

Now we'll run partial evaluation for different principals. TPE evaluates policies with:
- **Known**: Principal, action, and context
- **Unknown**: Specific resource (we only know the resource type)

Cedar returns a **residual policy** that describes what must be true about *any* resource for access to be permitted.

### Example 1: Kate (Customer)

Kate is a customer in the `custco` tenant. Let's see what residual policy Cedar produces:


In [ ]:
# Run partial evaluation for Kate
result = subprocess.run([
    'node', 'src/tpe/partial-eval.js',
    '--principal', 'Platform::Customer::"kate"',
    '--action', 'Platform::Action::"view"',
    '--resource-type', 'Platform::Chunk',
    '--out', 'out/residual-kate.json'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

# Load and display the residual
with open('out/residual-kate.json', 'r') as f:
    residual_kate = json.load(f)

print("\n=== Residual Policy for Kate ===\n")
print(f"Decision: {residual_kate['decision']}")
print(f"Principal: {residual_kate['principal']}")
print(f"Action: {residual_kate['action']}")
print(f"Resource Type: {residual_kate['resourceType']}")
print(f"\nResidual Policies ({len(residual_kate['residuals'])}):\n")

for i, residual in enumerate(residual_kate['residuals'], 1):
    print(f"{i}. Policy ID: {residual['id']} ({residual['type']})")
    print(f"   Condition: {residual['text'].split('when {')[1].split('}')[0].strip() if 'when {' in residual['text'] else 'N/A'}")
    print()


Loading Cedar schema, policies, and entities...
  Schema: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/schema.cedarschema
  Policies: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/policies
  Entities: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/entities.json

Performing partial evaluation:
  Principal: Platform::Customer::"kate"
  Action: Platform::Action::"view"
  Resource Type: Platform::Chunk

Attempting to use Cedar CLI for type-aware partial evaluation (TPE)...
✓ Successfully used Cedar CLI for type-aware partial evaluation
  Decision: UNKNOWN
  Residual policies: 4
✓ Residual policy written to out/residual-kate.json


=== Residual Policy for Kate ===

Decision: UNKNOWN
Principal: Platform::Customer::"kate"
Action: Platform::Action::"view"
Resource Type: Platform::Chunk

Residual Policies (4):

1. Policy ID: customer-view (permit)
   Condition: [Platform::Team::"custco-readers"].contains(resource.customer_readers_team)

2. Policy ID: tenant-scop

**Understanding Kate's Residual:**

1. **`tenant-scope` (permit)**: Resource must be in `custco` tenant
2. **`customer-view` (permit)**: Resource must be shared with `custco-readers` team
3. **`classification-limit` (forbid)**: Resource must NOT be `confidential`
4. **`employee-view` (permit)**: `false` - ignored (Kate is not an employee)

This means Kate can only access chunks that:
- Are in the `custco` tenant
- Are shared with the `custco-readers` team
- Are NOT classified as `confidential`

### Example 2: Alice (Employee)

Alice is an employee in the `custco` tenant. Let's see her residual:


In [ ]:
# Run partial evaluation for Alice
result = subprocess.run([
    'node', 'src/tpe/partial-eval.js',
    '--principal', 'Platform::Employee::"alice"',
    '--action', 'Platform::Action::"view"',
    '--resource-type', 'Platform::Chunk',
    '--out', 'out/residual-alice.json'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

# Load and display the residual
with open('out/residual-alice.json', 'r') as f:
    residual_alice = json.load(f)

print("\n=== Residual Policy for Alice ===\n")
print(f"Decision: {residual_alice['decision']}")
print(f"Principal: {residual_alice['principal']}")
print(f"\nResidual Policies ({len(residual_alice['residuals'])}):\n")

for i, residual in enumerate(residual_alice['residuals'], 1):
    print(f"{i}. Policy ID: {residual['id']} ({residual['type']})")
    # Extract condition from residual text
    if 'when {' in residual['text']:
        condition = residual['text'].split('when {')[1].split('}')[0].strip()
        print(f"   Condition: {condition}")
    else:
        print(f"   Condition: {residual['text']}")
    print()


Loading Cedar schema, policies, and entities...
  Schema: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/schema.cedarschema
  Policies: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/policies
  Entities: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/entities.json

Performing partial evaluation:
  Principal: Platform::Employee::"alice"
  Action: Platform::Action::"view"
  Resource Type: Platform::Chunk

Attempting to use Cedar CLI for type-aware partial evaluation (TPE)...
✓ Successfully used Cedar CLI for type-aware partial evaluation
  Decision: UNKNOWN
  Residual policies: 4
✓ Residual policy written to out/residual-alice.json


=== Residual Policy for Alice ===

Decision: UNKNOWN
Principal: Platform::Employee::"alice"

Residual Policies (4):

1. Policy ID: employee-view (permit)
   Condition: [Platform::Team::"custco-employees"].contains(resource.employee_readers_team)

2. Policy ID: customer-view (permit)
   Condition: false

3. Policy ID: tenant-scop

**Understanding Alice's Residual:**

1. **`tenant-scope` (permit)**: Resource must be in `custco` tenant
2. **`employee-view` (permit)**: Resource must be shared with `custco-employees` team
3. **`customer-view` (permit)**: `false` - ignored (Alice is not a customer)
4. **`classification-limit` (forbid)**: `false` - ignored (this forbid only applies to customers)

**Key difference from Kate:** Alice can access confidential resources! The `classification-limit` forbid doesn't apply to employees.

### Example 3: Mallory (Cross-Tenant Test)

Mallory is a customer in the `otherco` tenant. This demonstrates tenant isolation:


In [ ]:
# Run partial evaluation for Mallory
result = subprocess.run([
    'node', 'src/tpe/partial-eval.js',
    '--principal', 'Platform::Customer::"mallory"',
    '--action', 'Platform::Action::"view"',
    '--resource-type', 'Platform::Chunk',
    '--out', 'out/residual-mallory.json'
], capture_output=True, text=True)

print(result.stdout)

# Load and display the residual
with open('out/residual-mallory.json', 'r') as f:
    residual_mallory = json.load(f)

print("\n=== Residual Policy for Mallory ===\n")
print(f"Principal: {residual_mallory['principal']}")
print(f"\nResidual Policies ({len(residual_mallory['residuals'])}):\n")

for i, residual in enumerate(residual_mallory['residuals'], 1):
    print(f"{i}. Policy ID: {residual['id']} ({residual['type']})")
    if 'when {' in residual['text']:
        condition = residual['text'].split('when {')[1].split('}')[0].strip()
        print(f"   Condition: {condition}")
    print()

print("\n💡 Note: Since all chunks in the demo are in 'custco' tenant,")
print("   Mallory (in 'otherco' tenant) will have zero access.")
print("   This demonstrates tenant isolation working correctly!")


Loading Cedar schema, policies, and entities...
  Schema: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/schema.cedarschema
  Policies: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/policies
  Entities: /Users/pjw/Dropbox/prog/authz/cedar-rag-authz-demo/cedar/entities.json

Performing partial evaluation:
  Principal: Platform::Customer::"mallory"
  Action: Platform::Action::"view"
  Resource Type: Platform::Chunk

Attempting to use Cedar CLI for type-aware partial evaluation (TPE)...
✓ Successfully used Cedar CLI for type-aware partial evaluation
  Decision: UNKNOWN
  Residual policies: 4
✓ Residual policy written to out/residual-mallory.json


=== Residual Policy for Mallory ===

Principal: Platform::Customer::"mallory"

Residual Policies (4):

1. Policy ID: customer-view (permit)
   Condition: [Platform::Team::"otherco-readers"].contains(resource.customer_readers_team)

2. Policy ID: tenant-scope (permit)
   Condition: Platform::Tenant::"otherco" == (resource.ten

## Step 3: Compiling Residuals to OpenSearch Filters

Now we translate the Cedar residual policies into OpenSearch query filters. This is **application logic** - Cedar doesn't generate database queries, but tells us which attributes matter.

The compiler:
1. Parses the residual policy conditions
2. Maps Cedar expressions to OpenSearch filter clauses
3. Handles `permit` policies (→ `must` clauses)
4. Handles `forbid` policies (→ `must_not` clauses)

### Compile Kate's Residual


In [ ]:
# Compile Kate's residual to OpenSearch filter
result = subprocess.run([
    'node', 'src/compile/residual-to-filter.js',
    '--residual', 'out/residual-kate.json',
    '--out', 'examples/queries/opensearch-filter-kate.json'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

# Load and display the compiled filter
with open('examples/queries/opensearch-filter-kate.json', 'r') as f:
    filter_kate = json.load(f)

print("\n=== OpenSearch Filter for Kate ===\n")
print(json.dumps(filter_kate['filter'], indent=2))


Loading residual policy from out/residual-kate.json...

Compiling residual to OpenSearch filter...
Residual structure: {
  "decision": "UNKNOWN",
  "principal": "Platform::Customer::\"kate\"",
  "action": "Platform::Action::\"view\"",
  "resourceType": "Platform::Chunk",
  "residuals": [
    {
      "id": "customer-vi...

✓ OpenSearch filter written to examples/queries/opensearch-filter-kate.json

Filter structure:
{
  "bool": {
    "must": [
      {
        "term": {
          "customer_readers_team_id": "custco-readers"
        }
      },
      {
        "term": {
          "tenant_id": "custco"
        }
      }
    ],
    "must_not": [
      {
        "term": {
          "classification": "confidential"
        }
      }
    ]
  }
}


=== OpenSearch Filter for Kate ===

{
  "bool": {
    "must": [
      {
        "term": {
          "customer_readers_team_id": "custco-readers"
        }
      },
      {
        "term": {
          "tenant_id": "custco"
        }
      }
    ],
    

**Kate's Filter Breakdown:**

- **`must`** (all must be true):
  - `tenant_id = "custco"` (from `tenant-scope` policy)
  - `customer_readers_team_id = "custco-readers"` (from `customer-view` policy)

- **`must_not`** (must be false):
  - `classification ≠ "confidential"` (from `classification-limit` forbid policy)

This filter ensures only authorized chunks can be retrieved for Kate.

### Compile Alice's Residual


In [ ]:
# Compile Alice's residual to OpenSearch filter
result = subprocess.run([
    'node', 'src/compile/residual-to-filter.js',
    '--residual', 'out/residual-alice.json',
    '--out', 'examples/queries/opensearch-filter-alice.json'
], capture_output=True, text=True)

print(result.stdout)

# Load and display the compiled filter
with open('examples/queries/opensearch-filter-alice.json', 'r') as f:
    filter_alice = json.load(f)

print("\n=== OpenSearch Filter for Alice ===\n")
print(json.dumps(filter_alice['filter'], indent=2))


Loading residual policy from out/residual-alice.json...

Compiling residual to OpenSearch filter...
Residual structure: {
  "decision": "UNKNOWN",
  "principal": "Platform::Employee::\"alice\"",
  "action": "Platform::Action::\"view\"",
  "resourceType": "Platform::Chunk",
  "residuals": [
    {
      "id": "employee-v...

✓ OpenSearch filter written to examples/queries/opensearch-filter-alice.json

Filter structure:
{
  "bool": {
    "must": [
      {
        "term": {
          "employee_readers_team_id": "custco-employees"
        }
      },
      {
        "term": {
          "tenant_id": "custco"
        }
      }
    ]
  }
}


=== OpenSearch Filter for Alice ===

{
  "bool": {
    "must": [
      {
        "term": {
          "employee_readers_team_id": "custco-employees"
        }
      },
      {
        "term": {
          "tenant_id": "custco"
        }
      }
    ]
  }
}


**Alice's Filter Breakdown:**

- **`must`**:
  - `tenant_id = "custco"`
  - `employee_readers_team_id = "custco-employees"`

- **`must_not`**: (empty - no classification restriction!)

**Key difference:** Alice has no `must_not` clause for classification because employees can access confidential resources.

### Compile Mallory's Residual


### Validating OpenSearch Filters

Now let's validate that our compiled filters have correct OpenSearch syntax:


In [ ]:
# Validate all compiled filters
filter_files = [
    'examples/queries/opensearch-filter-kate.json',
    'examples/queries/opensearch-filter-alice.json',
    'examples/queries/opensearch-filter-mallory.json'
]

print("=== Validating OpenSearch Filters ===\n")

for filter_file in filter_files:
    print(f"Validating {filter_file}...")
    result = subprocess.run([
        'node', 'src/compile/validate-filter.js', filter_file
    ], capture_output=True, text=True)
    
    print(result.stdout)
    if result.returncode != 0:
        print(f"❌ Validation failed for {filter_file}")
        if result.stderr:
            print(result.stderr)
    print()


In [ ]:
# Compile Mallory's residual to OpenSearch filter
result = subprocess.run([
    'node', 'src/compile/residual-to-filter.js',
    '--residual', 'out/residual-mallory.json',
    '--out', 'examples/queries/opensearch-filter-mallory.json'
], capture_output=True, text=True)

print(result.stdout)

# Load and display the compiled filter
with open('examples/queries/opensearch-filter-mallory.json', 'r') as f:
    filter_mallory = json.load(f)

print("\n=== OpenSearch Filter for Mallory ===\n")
print(json.dumps(filter_mallory['filter'], indent=2))

print("\n💡 This filter requires 'otherco' tenant, but all chunks are in 'custco'.")
print("   Result: Mallory has zero access (tenant isolation enforced).")


Loading residual policy from out/residual-mallory.json...

Compiling residual to OpenSearch filter...
Residual structure: {
  "decision": "UNKNOWN",
  "principal": "Platform::Customer::\"mallory\"",
  "action": "Platform::Action::\"view\"",
  "resourceType": "Platform::Chunk",
  "residuals": [
    {
      "id": "customer...

✓ OpenSearch filter written to examples/queries/opensearch-filter-mallory.json

Filter structure:
{
  "bool": {
    "must": [
      {
        "term": {
          "customer_readers_team_id": "otherco-readers"
        }
      },
      {
        "term": {
          "tenant_id": "otherco"
        }
      }
    ],
    "must_not": [
      {
        "term": {
          "classification": "confidential"
        }
      }
    ]
  }
}


=== OpenSearch Filter for Mallory ===

{
  "bool": {
    "must": [
      {
        "term": {
          "customer_readers_team_id": "otherco-readers"
        }
      },
      {
        "term": {
          "tenant_id": "otherco"
        }
      

## Step 4: Comparing Authorization Scopes

Let's compare what each principal can access:


In [11]:
# Load expected scopes
principals = ['kate', 'alice', 'mallory']
scopes = {}

for principal in principals:
    scope_file = f'examples/expected/{principal}-scope.json'
    if Path(scope_file).exists():
        with open(scope_file, 'r') as f:
            scopes[principal] = json.load(f)

# Display comparison
print("=== Authorization Scope Comparison ===\n")

for principal, scope in scopes.items():
    print(f"## {principal.title()} ({scope['principal']})")
    print(f"\n{scope['description']}")
    print(f"\nFilter Conditions: {json.dumps(scope['filter_conditions'], indent=2)}")
    print(f"Authorized Resources: {len(scope['authorized_resources'])}")
    if scope['authorized_resources']:
        for resource in scope['authorized_resources']:
            print(f"  - {resource['id']} (tenant: {resource['tenant']}, classification: {resource.get('classification', 'N/A')})")
    else:
        print("  (none - no access)")
    print()


=== Authorization Scope Comparison ===

## Kate (Platform::Customer::"kate")

Kate (customer) can access chunks in custco tenant that are shared with custco-readers team and are not confidential

Filter Conditions: {
  "tenant": "custco",
  "customer_readers_team": "custco-readers",
  "classification_not": "confidential"
}
Authorized Resources: 1
  - q3-plan#1 (tenant: custco, classification: internal)

## Alice (Platform::Employee::"alice")

Alice (employee) can access all chunks in custco tenant that are shared with custco-employees team, including confidential ones

Filter Conditions: {
  "tenant": "custco",
  "employee_readers_team": "custco-employees"
}
Authorized Resources: 2
  - q3-plan#1 (tenant: custco, classification: internal)
  - hr-note#1 (tenant: custco, classification: confidential)

## Mallory (Platform::Customer::"mallory")

Mallory (customer in otherco tenant) can only access chunks in the otherco tenant that are shared with otherco-readers team and are not confidenti

## Step 5: Understanding the Complete Flow

Here's how authorization-scoped retrieval works in a RAG system:

```
┌─────────────────┐
│  User Query     │
│  (e.g., Kate)   │
└────────┬────────┘
         │
         ▼
┌─────────────────────────────────┐
│  Cedar TPE                      │
│  - Known: Principal, Action     │
│  - Unknown: Resource            │
│  → Residual Policy              │
└────────┬────────────────────────┘
         │
         ▼
┌─────────────────────────────────┐
│  Compile Residual               │
│  → OpenSearch Filter            │
└────────┬────────────────────────┘
         │
         ▼
┌─────────────────────────────────┐
│  Vector Search                  │
│  - Query: User's question       │
│  - Filter: Authorization scope  │
│  → Only authorized chunks       │
└────────┬────────────────────────┘
         │
         ▼
┌─────────────────────────────────┐
│  LLM Context Assembly           │
│  - Only authorized data         │
│  → Safe to send to model        │
└─────────────────────────────────┘
```

**Key Guarantees:**

1. ✅ **Authorization happens before retrieval** - not in the prompt
2. ✅ **The model never sees unauthorized data** - it's filtered out
3. ✅ **Policy changes automatically update retrieval** - no code changes needed
4. ✅ **Multi-tenant isolation** - enforced at the database query level

## Summary

This demo showed:

1. **Cedar TPE** produces residual policies that describe authorized resources
2. **Residual compilation** translates Cedar conditions into database filters
3. **Different principals** get different authorization scopes (Kate vs Alice vs Mallory)
4. **Tenant isolation** is enforced automatically (Mallory has no access to custco resources)

The pattern ensures that **authorization is enforced before retrieval**, making it impossible for unauthorized data to leak into LLM context.


## Next Steps

To extend this demo:

1. **Add more principals** - Create new entities and see how residuals change
2. **Modify policies** - Update policies and observe how filters change
3. **Connect to OpenSearch** - Use the compiled filters with actual vector search
4. **Test edge cases** - Try different actions, contexts, or resource types

For more details, see the [README.md](README.md) file.
